In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_key = os.getenv("TAVILY_API_KEY")

In [9]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]


ToolNode로 도구 노드 구축

In [ ]:

from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results = 2)
tools = [tool]
tool_node = ToolNode(tools)

In [10]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """ A node that runs the tools requested in the last AIMessage. """

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No messages found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(ToolMessage(tool_call.id, tool_call.name, tool_result))
        return {"messages": outputs}

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}


In [13]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

인터넷 검색이 필요한 질문

In [15]:
graph.invoke({"messages": {"role": "user", "content": "지금 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='지금 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='a8930f4c-37fb-402d-bd47-ce7976f6ded4'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IzozYlDiwQ7juPILM20tpB4c', 'function': {'arguments': '{"query":"current president of South Korea"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 87, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ff77111d-7947-4a10-8b7f-56b499dad8eb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current president of South Korea'}, 'id': 'call_Izo

LLM이 답할 수 있는 질문

In [16]:
graph.invoke({"messages": {"role": "user", "content": "너가 대답하지 못하고 인터넷 검색이 필요한 질문 3개를 예시로 제시해"}})

{'messages': [HumanMessage(content='너가 대답하지 못하고 인터넷 검색이 필요한 질문 3개를 예시로 제시해', additional_kwargs={}, response_metadata={}, id='e8714c55-a97d-40a3-99c0-fe129fb03689'),
  AIMessage(content='다음은 제가 대답하기 어려운 질문 3가지 예시입니다:\n\n1. 현재 날씨 정보: 특정 지역의 현재 날씨가 어떻게 되는지에 대한 질문.\n2. 최신 뉴스: 최근의 사건이나 뉴스에 대한 정보, 예를 들면 "오늘 어떤 주요 뉴스가 있나요?"\n3. 주식 시장 동향: 현재 주식 시장의 동향이나 특정 기업의 주가에 대한 질문.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 101, 'total_tokens': 193, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-c40a5c73-f873-446f-af3c-9055d55bfb33-0', usage_metadata={'input_tokens': 101, 'output_tokens': 92, 'total_tokens': 193, 'input_token_details': {'audio': 0,